# Исследование музыки больших городов

Существует догадка, что музыкальные предпочтения людей в Москве и Санкт-Петербурге отличаются, это необходимо проверить.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели.
2. В разные дни недели преобладают разные жанры, причём для каждого города своя статистика.
3. Москва и Петербург различаются по предпочтениям жанров музыки.

**Описание данных**</br>
Данные о пользователях предоставлены из сервиса Яндекс Музыка и находятся в файле `yandex_music_project.csv`.

**Ход исследования**</br>
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Импортируем библиотеку `pandas` для работы

In [1]:
import pandas as pd # импорт библиотеки pandas

Получим файл `yandex_music_project.csv`

In [2]:
df = pd.read_csv('yandex_music_project.csv')

Выведем на экран первый 10 строк таблицы:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Получим информацию о таблице с помощью `info()`:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.

Подробно разберём то, что находится в столбцах:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных


Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков
Изучим названия столбцов:

In [5]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

*Заметно, что столбцы имеют несоответствие представлениям о хорошем стиле. Исправим это:*
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
df = df.rename(columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

Изучим результат преобразования:

In [7]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

Изучим, сколько пропусков в таблице. Используем `isna()` и `sum()`:

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем, нет конкретного исполнителя. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину. Этот проект предусматривает упрощенное решение вопроса.

Заменяем пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [9]:
columns_to_replace = ['track', 'artist', 'genre']# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
for el in columns_to_replace:
    df[el] = df[el].fillna('unknown')

Проверим, есть ли пропуски:

In [10]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

Подсчитаем дубликаты:

In [11]:
df.duplicated().sum()# подсчёт явных дубликатов

3826

Удалим явные дубликаты:

In [12]:
df = df.drop_duplicates()

Убедимся, что дубликатов нет:

In [13]:
df.duplicated().sum()

0

Теперь избавимся от неявных дубликатов в колонке `genre`. 

Выведем на экран список уникальных названий жанров. 

In [14]:
df.loc[:, 'genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

На первй взгляд, есть неявные дубликаты только у названия `hiphop`:
* *hip*,
* *hop*,
* *hip-hop*.

Для очистки от них таблицы используем метод `replace()` с двумя аргументами: списком строк-дубликатов (включащий *hip*, *hop* и *hip-hop*) и строкой с правильным значением. 

In [15]:
bad_name = ['hip', 'hop', 'hip-hop']# Устранение неявных дубликатов
for el in bad_name:
    df['genre'] = df['genre'].replace(el, 'hiphop')

Изучим результат преобразования

In [16]:
df.loc[:, 'genre'].sort_values().unique()# Проверка на неявные дубликаты

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.
Пропущенные значения заменили на `'unknown'`.
Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Сгруппируем данные по городу и посчитайте прослушивания в каждой группе.

In [17]:
df.groupby('city')['track'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

Вероятно, пользователей в Петербурге больше, чем в Москве.

Теперь сгруппируем данные по дню недели.

In [18]:
df.groupby('day')['track'].count()# Подсчёт прослушиваний в каждый из трёх дней

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

*В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.*

Напишем функцию, которая объединит группировку по городу и по дням недели. 
Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города.

In [19]:
def number_tracks(day, city):
    """Функция для подсчёта прослушиваний для конкретного города и дня.
    """
    track_list = df.loc[df.loc[:, 'day'] == day]
    track_list = track_list.loc[track_list.loc[:, 'city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [20]:
number_tracks('Monday','Moscow')# количество прослушиваний в Москве по понедельникам

15740

In [21]:
number_tracks('Monday','Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по понедельникам

5614

In [22]:
number_tracks('Wednesday','Moscow')# количество прослушиваний в Москве по средам

11056

In [23]:
number_tracks('Wednesday','Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по средам

7003

In [24]:
number_tracks('Friday','Moscow')# количество прослушиваний в Москве по пятницам

15945

In [25]:
number_tracks('Friday','Saint-Petersburg')# количество прослушиваний в Санкт-Петербурге по пятницам

5895

Занесём количество прослушиваний по дням и городам в таблицу, испльзуем конструктор `pd.DataFrame`.

In [26]:
columns = ['city', 'monday', 'wednesday', 'friday']# Таблица с результатами
data = [['Moscow', '15740', '11056', '15945'],
        ['Saint-Petersburg', '5614', '7003', '5895']]
cities_musical_activity = pd.DataFrame(columns = columns, data= data)
cities_musical_activity

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [27]:
moscow_general = df[df['city'] == 'Moscow']

In [28]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()`, которая возвращает информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [29]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [30]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [31]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [32]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [33]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитатем прослушивания треков каждого жанра методом `count()`. Проведём сортировку.

In [34]:
moscow_genres = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_genres.sort_values(ascending = False)

Выведем на экран первые десять строк `moscow_genres`:

In [35]:
moscow_genres.head(10)# просмотр первых 10 строк moscow_genres

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Повторим для Петербурга

In [36]:
spb_genres = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_genres.sort_values(ascending = False)

Выведем на экран первые десять строк:

In [37]:
spb_genres.head(10)# просмотр первых 10 строк spb_genres

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Однако, относительно среды между городами наблюдается различие: Москва больше слушает в понедельник и пятницу, а Петербург наоборот — больше в среду, но меньше в понедельник и пятницу.

Были проверены три гипотезы, установлено:
1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 
2. Музыкальные предпочтения не сильно меняются в течение недели. Небольшие различия заметны в начале недели, по понедельникам:
    - в Москве слушают музыку жанра “world”,
    - в Петербурге — джаз и классику.
3. Во вкусах пользователей столиц больше общего чем различий. 

**В итоге:**
1. Первая гипотеза полностью подтвердилась.
2. Вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.
3. Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 